In [7]:
import pandas as pd
from sklearn import model_selection
import numpy as np
import sklearn.metrics
from sklearn.model_selection import train_test_split  
import lightgbm as lgb

In [8]:
print('开始处理特征...')
train_path = 'data/train_and_test/train.csv'
test_path = 'data/train_and_test/test.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
used_feature = [ 'create_count', 'create_day_mean', 'create_day_max',
       'create_day_min', 'create_day_std', 'create_day_var',
       'last_day_cut_max_day', 'create_sub_register_x',
       'max_create_times_per_day', 'device_type', 'register_type',
       'register_day_cut_max_day', 'launch_count', 'launch_day_mean',
       'launch_day_max', 'launch_day_min', 'launch_day_std', 'launch_day_var',
       'launch_day_cut_max_day', 'create_sub_register_y',
       'max_launch_times_per_day', 'activity_count', 'activity_day_mean',
       'activity_day_max', 'activity_day_min', 'activity_day_std',
       'activity_day_var', 'activity_day_ske', 'activity_day_kur',
       'activity_day_cut_max_day', 'create_sub_register', '0_page_count',
       '1_page_count', '2_page_count', '3_page_count', '4_page_count',
       '0_page_count_div_sum', '1_page_count_div_sum', '2_page_count_div_sum',
       '3_page_count_div_sum', '4_page_count_div_sum', '0_action_count',
       '1_action_count', '2_action_count', '3_action_count', '4_action_count',
       '5_action_count', '0_action_count_div_sum', '1_action_count_div_sum',
       '2_action_count_div_sum', '3_action_count_div_sum',
       '4_action_count_div_sum', '5_action_count_div_sum',
       'max_act_times_per_day', 'watch_self']
used_feature = np.array(used_feature)
train_feature = train[used_feature]
test_feature = test[used_feature]
label = train['label']
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(train_feature, label, test_size=0.2,random_state=1017)


开始处理特征...


In [9]:
print('载入数据......')
lgb_train = lgb.Dataset(train_feature, label)
lgb_eval = lgb.Dataset(X_test, Y_test, reference=lgb_train)
print('开始训练......')
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'auc', 'binary_logloss'}
}
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=58,
                valid_sets=lgb_eval
                )
gbm.save_model('model/lgb_model.txt')

temp = gbm.predict(X_test)
temp[temp>0.5]=1
temp[temp<0.5]=0
print('结果：' + str(sklearn.metrics.f1_score(Y_test, temp)))
print('特征重要性：'+ str(list(gbm.feature_importance())))

pre = gbm.predict(test_feature)
df_result = pd.DataFrame()
df_result['user_id'] = test['user_id']
df_result['result'] = pre
df_result.to_csv('result/lgb_result.csv', index=False)
pre[pre >= 0.5]=1
pre[pre < 0.5]=0
pre = map(int,pre)
print('为1的个数：' + str(len(np.where(np.array(pre)==1)[0])))
print('为0的个数：' + str(len(np.where(np.array(pre)==0)[0])))

载入数据......
开始训练......
[1]	valid_0's auc: 0.871164	valid_0's binary_logloss: 0.651163
[2]	valid_0's auc: 0.873073	valid_0's binary_logloss: 0.618094
[3]	valid_0's auc: 0.87408	valid_0's binary_logloss: 0.590797
[4]	valid_0's auc: 0.875177	valid_0's binary_logloss: 0.567904
[5]	valid_0's auc: 0.875476	valid_0's binary_logloss: 0.548641
[6]	valid_0's auc: 0.876429	valid_0's binary_logloss: 0.532336
[7]	valid_0's auc: 0.876828	valid_0's binary_logloss: 0.518524
[8]	valid_0's auc: 0.877073	valid_0's binary_logloss: 0.506741
[9]	valid_0's auc: 0.877317	valid_0's binary_logloss: 0.496703
[10]	valid_0's auc: 0.877455	valid_0's binary_logloss: 0.488114
[11]	valid_0's auc: 0.877777	valid_0's binary_logloss: 0.480584
[12]	valid_0's auc: 0.877935	valid_0's binary_logloss: 0.474262
[13]	valid_0's auc: 0.878104	valid_0's binary_logloss: 0.468724
[14]	valid_0's auc: 0.878421	valid_0's binary_logloss: 0.463773
[15]	valid_0's auc: 0.878576	valid_0's binary_logloss: 0.459558
[16]	valid_0's auc: 0.87873	

In [10]:
lgb = pd.read_csv('result/lgb_result.csv')
res = lgb[lgb['result'] >= 0.5]
print(len(res))
del res['result']
res.to_csv('result2/dealed_result.csv', index=False, header=False)

21888
